In [1]:
import numpy as np
import os
import tensorflow as tf

In [2]:
tokens = np.array(()) # empty numpy array
for filename in os.listdir('Dataset_tokenized_BPE'): # iterate through all files in the folder
    if filename.endswith('.npy'): # if the file is a .npy file
        for i in range(len(np.load(('Dataset_tokenized_BPE/' + filename), allow_pickle=True)[()].get('tokens'))): # iterate through all the tokens in the file
            tokens = np.append(tokens, np.load(('Dataset_tokenized_BPE/' + filename), allow_pickle=True)[()].get('tokens')[i]) # append the tokens to the numpy array
        continue 
    else: 
        continue 

In [3]:
print("smallest token:{}, biggest token:{}, number of tokens:{}, missing tokens:{}".format(np.min(tokens),np.max(tokens),np.unique(tokens).shape,set(range(0, 500))-set(tokens))) # print some information about the tokens
vocab_size = int(np.max(tokens))
data = tf.cast(tokens, tf.int32) # put tokens into a tensor and cast to int32
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n] # first 90% of data
val_data = data[n:] # last 10% of data

smallest token:1.0, biggest token:499.0, number of tokens:(493,), missing tokens:{0, 176, 178, 180, 182, 89, 186}


In [4]:
BLOCK_SIZE = 12
BATCH_SIZE = 2
    
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = tf.random.uniform((BATCH_SIZE,), maxval=len(data)-BLOCK_SIZE, dtype=tf.int32)
    inputs = tf.stack([data[i:i+BLOCK_SIZE] for i in ix])
    targets = tf.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    return inputs, targets

get_batch('train')

(<tf.Tensor: shape=(2, 12), dtype=int32, numpy=
 array([[225, 332, 317,  49, 470, 277,  48, 387,  46, 316,  52, 480],
        [204,  57, 411,  58, 226, 208,  57, 317,  65, 254, 214,  56]])>,
 <tf.Tensor: shape=(2, 12), dtype=int32, numpy=
 array([[332, 317,  49, 470, 277,  48, 387,  46, 316,  52, 480,  53],
        [ 57, 411,  58, 226, 208,  57, 317,  65, 254, 214,  56, 238]])>)

In [6]:
moin = get_batch('train') # tuple of two tensors (inputs, targets) of shape (batch_size, block_size)
print(moin)
inputs, targets = moin # unpack tuple

layer = tf.keras.layers.Embedding(input_dim=vocab_size, 
                          output_dim=512, # each token gets a 512-vector embedding
                          input_length=BLOCK_SIZE)
layer(inputs).shape

(<tf.Tensor: shape=(2, 12), dtype=int32, numpy=
array([[187,  52, 225, 189,  55, 246, 190,  58, 246, 191,  69, 225],
       [ 58, 463,  55, 369,  52, 238, 190,  53, 324,  53, 238, 198]])>, <tf.Tensor: shape=(2, 12), dtype=int32, numpy=
array([[ 52, 225, 189,  55, 246, 190,  58, 246, 191,  69, 225, 193],
       [463,  55, 369,  52, 238, 190,  53, 324,  53, 238, 198,  60]])>)


TensorShape([2, 12, 512])

In [65]:
def positional_enc

<tf.Tensor: shape=(10, 512), dtype=float32, numpy=
array([[-0.04916601, -0.02973673, -0.01249365, ..., -0.00418216,
         0.0386625 ,  0.02667138],
       [-0.03071208,  0.01060237,  0.02099904, ...,  0.03319449,
        -0.04411687,  0.04185689],
       [-0.03068695,  0.00648274,  0.03935588, ...,  0.03560025,
         0.02234754,  0.0326318 ],
       ...,
       [-0.0050758 ,  0.04384517,  0.01985851, ..., -0.04682778,
        -0.02236791,  0.03188093],
       [ 0.00323981,  0.03662967, -0.04555812, ..., -0.04969875,
        -0.04542772,  0.0441572 ],
       [-0.01620966, -0.0274914 ,  0.04703097, ..., -0.00346784,
         0.01711167, -0.04980812]], dtype=float32)>

In [45]:
d_model = 512 # also embedding size
d_v, d_q, d_k = 64, 64, 64
dff = 2048 # inner feedforward layer dim
n_heads = 8
n_layers = 6
dropout_rate = 0.1
batch_size = 64
seq_len = 50


TypeError: cannot unpack non-iterable int object

In [ ]:
# implementaion of dot product attention
class DotProductAttention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(DotProductAttention, self).__init__(**kwargs)
        
    def call(self, q, k, v, d_k, mask=None):
        
    

In [20]:
B,T,C = 4,8,32 # batch, time, channels
x = tf.random.normal((B,T,C))

head_size = 16
key = tf.keras.layers.Dense(units=head_size, use_bias=False)
query = tf.keras.layers.Dense(units=head_size, use_bias=False)
value = tf.keras.layers.Dense(units=head_size, use_bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)
scores = tf.matmul(q, tf.transpose(k, perm=[0, 2, 1])) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = tf.linalg.band_part(tf.ones((T, T)), -1, 0) # lower triangular matrix
scores = tf.where(tril == 0, tf.fill(tril.shape, -float('inf')), scores) # mask out the upper triangle
scores = tf.nn.softmax(scores, axis=-1) # (B, T, T)

v = value(x) # (B, T, 16) 
out = tf.matmul(scores, v) # (B, T, T) @ (B, T, 16) ---> (B, T, 16)

print(out.shape)

(4, 8, 16)


In [24]:
input_array = np.random.randint(1000, size=(32, 10))
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(1000, 64, input_length=10))
model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
print(output_array.shape)



1/1 [==============================] - 0s 211ms/step
(32, 10, 64)


In [28]:
tf.keras.layers.Embedding(input_dim=vocab_size, 
                          output_dim=d_model, 
                          input_length=block_size)

array([ 0.0286544 , -0.01695011, -0.01025267, -0.02617158, -0.00404854,
        0.04775255, -0.02500275,  0.0416499 ,  0.04432914,  0.0064167 ,
        0.02524245,  0.00858472,  0.03022489, -0.04360149, -0.04557949,
       -0.04291893,  0.02719354, -0.00238859,  0.01595623,  0.04848215,
        0.03093796, -0.03719582,  0.00072558, -0.03027285, -0.04864417,
        0.0171029 ,  0.03791413, -0.04314154,  0.01211748,  0.00926459,
       -0.03436134, -0.0265825 , -0.00529455,  0.03243819, -0.02915757,
        0.00714353, -0.04682566,  0.02050204, -0.04006963,  0.02837286,
        0.04162252, -0.02827531,  0.02999714, -0.03455625, -0.02379003,
        0.0294604 ,  0.00350038, -0.0319316 , -0.04055564, -0.03999837,
        0.00173689, -0.01042766,  0.0348827 , -0.03970977, -0.00249738,
        0.03113553, -0.04805944, -0.01816513,  0.00125686,  0.03857238,
       -0.00246302, -0.02902842, -0.01994125, -0.04502575], dtype=float32)

In [15]:
class SingleHeadAttention(tf.keras.layers.Layer):
    def __init__(self, head_size, **kwargs):
        super(SingleHeadAttention, self).__init__(**kwargs)
        self.key = tf.keras.layers.Dense(units=head_size, use_bias=False)
        self.query = tf.keras.layers.Dense(units=head_size, use_bias=False)
        self.value = tf.keras.layers.Dense(units=head_size, use_bias=False)
        
    def call(self, x, mask=None):
        k = self.key(x) # (B, T, head_size)
        

<tf.Tensor: shape=(8, 8), dtype=float32, numpy=
array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [3.2446999e-07, 9.9999964e-01, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [2.1704189e-07, 9.9002355e-01, 9.9762417e-03, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.8885803e-04, 9.7978640e-01, 2.0003926e-02, 2.0735459e-05,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [1.0849527e-01, 9.4697058e-02, 1.0691612e-01, 2.6824350e-02,
        6.6306716e-01, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [3.5939179e-02, 7.5873780e-01, 2.0459630e-01, 8.4528483e-06,
        7.1826362e-04, 6.8539862e-08, 0.0000000e+00, 0.0000000e+00],
       [9.9802607e-01, 9.1235660e-04, 3.6117420e-04, 1.7029686e-05,
        1.7039441e-04, 4.1312503e-04, 9.9868579e-05, 0.0000000